In [2]:
#! pip3 list | grep tensorflow 
! pip3 install --user tensorflow==2.4.0
! pip3 install --user ipywidgets nbconvert
!python -m pip install --user --upgrade pip
!pip3 install pandas scikit-learn keras tensorflow-datasets --user


     |████████████████████████████████| 394.7 MB 1.6 MB/s eta 0:00:01     |███                             | 37.7 MB 1.6 MB/s eta 0:03:48     |███▉                            | 47.4 MB 1.6 MB/s eta 0:03:42     |███████▎                        | 90.2 MB 1.7 MB/s eta 0:03:03               | 122.1 MB 1.6 MB/s eta 0:02:54      | 144.6 MB 1.6 MB/s eta 0:02:40  | 202.0 MB 1.6 MB/s eta 0:02:04     |████████████████▋               | 205.3 MB 1.6 MB/s eta 0:02:01   | 239.5 MB 1.6 MB/s eta 0:01:40�████████▉           | 257.0 MB 1.4 MB/s eta 0:01:40�█████████           | 259.6 MB 1.4 MB/s eta 0:01:39     |█████████████████████▎          | 261.8 MB 1.4 MB/s eta 0:01:37     |██████████████████████▎         | 274.1 MB 1.4 MB/s eta 0:01:28��████████████████▌       | 302.6 MB 1.5 MB/s eta 0:01:03     |██████████████████████████▎     | 323.8 MB 1.5 MB/s eta 0:00:49 | 333.5 MB 1.5 MB/s eta 0:00:42Killed
  Using cached ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
  Using cached jupyterlab_widgets-1.0.0

In [3]:
TRAINER_FILE = "tfjobheart.py"
KUBERNETES_FILE = "tfjob-heartdisease.yaml"


In [4]:

import re

from IPython.utils.capture import CapturedIO


def get_resource(captured_io: CapturedIO) -> str:
    """
    Gets a resource name from `kubectl apply -f <configuration.yaml>`.

    :param str captured_io: Output captured by using `%%capture` cell magic
    :return: Name of the Kubernetes resource
    :rtype: str
    :raises Exception: if the resource could not be created
    """
    out = captured_io.stdout
    matches = re.search(r"^(.+)\s+created", out)
    if matches is not None:
        return matches.group(1)
    else:
        raise Exception(f"Cannot get resource as its creation failed: {out}. It may already exist.")



In [8]:
import pandas as  pd
#data = pd.read_csv("heart.csv")
#data.head()
file ="heart.csv"
df= pd.read_csv(file)


In [13]:
%%writefile $TRAINER_FILE
import argparse
import logging
import json
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

from numpy.random import seed

import tensorflow as tf
tf.random.set_seed(221)
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

logging.getLogger().setLevel(logging.INFO)

def make_datasets_unbatched():
    data = pd.read_csv("heart.csv")
    data.head()
    
    data.apply(lambda x: sum(x.isnull()),axis=0)
    
    # List of variables with missing values
    
    vars_with_na=[var for var in data.columns if data[var].isnull().sum()>1]
#remove outliers
    
    def removeOutlier(att, data):
        lowerbound = att.mean() - 3 * att.std()
        upperbound = att.mean() + 3 * att.std()
        #print('lowerbound: ',lowerbound,' -------- upperbound: ', upperbound )
        df1 = data[(att > lowerbound) & (att < upperbound)]
        #print((data.shape[0] - df1.shape[0]), ' number of outliers from ', data.shape[0] )
        #print(' ******************************************************')
        data = df1.copy()
        return data
    data = removeOutlier(data.trtbps, data)
    data = removeOutlier(data.chol, data)
    
    #resampling
    from sklearn.utils import resample
    
    # Separate Target Classes
    df_1 = data[data.output==1]
    df_2 = data[data.output==0]
    
    # Upsample minority class
    df_upsample_1 = resample(df_2, 
     replace=True,     # sample with replacement
                                 n_samples=163,    # to match majority class
                                 random_state=123) # reproducible results
    # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_1, df_upsample_1])
    
    # Display new class counts
    df_upsampled.output.value_counts()

    x = df_upsampled.drop('output', axis = 1)
    y = df_upsampled['output']
    
    #Split dataset

    
    x_train,x_test, y_train, y_test = tts(x,y, test_size = 0.2, random_state = 111)
    
    #Scaling
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)
    
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    train = train_dataset.cache().shuffle(2000).repeat()
    return train, test_dataset

def model(args):
    seed(1)
    model = Sequential()
    model.add(Dense(10, activation='relu', input_dim=13))
    #model.add(BatchNormalization())
    model.add(Dense(10, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    opt = args.optimizer
    model.compile(optimizer=opt,
                loss='binary_crossentropy',
                metrics=['accuracy'])
    tf.keras.backend.set_value(model.optimizer.learning_rate, args.learning_rate)
    return model


def main(args):
    # MultiWorkerMirroredStrategy creates copies of all variables in the model's
    # layers on each device across all workers
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy(
        communication=tf.distribute.experimental.CollectiveCommunication.AUTO)
    logging.debug(f"num_replicas_in_sync: {strategy.num_replicas_in_sync}")
    BATCH_SIZE_PER_REPLICA = args.batch_size
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
    
    # Datasets need to be created after instantiation of `MultiWorkerMirroredStrategy`
    train_dataset, test_dataset = make_datasets_unbatched()
    train_dataset = train_dataset.batch(batch_size=BATCH_SIZE)
    test_dataset = test_dataset.batch(batch_size=BATCH_SIZE)

    # See: https://www.tensorflow.org/api_docs/python/tf/data/experimental/DistributeOptions
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = \
    tf.data.experimental.AutoShardPolicy.DATA
    
    train_datasets_sharded  = train_dataset.with_options(options)
    test_dataset_sharded = test_dataset.with_options(options)
    
    with strategy.scope():
        # Model building/compiling need to be within `strategy.scope()`.
        multi_worker_model = model(args)
        # Keras' `model.fit()` trains the model with specified number of epochs and
        # number of steps per epoch. 
        multi_worker_model.fit(train_datasets_sharded,
                         epochs=50,
                         steps_per_epoch=30)

        eval_loss, eval_acc = multi_worker_model.evaluate(test_dataset_sharded, 
                                                    verbose=0, steps=10)
        # Log metrics for Katib
logging.info("loss={:.4f}".format(eval_loss))
        logging.info("accuracy={:.4f}".format(eval_acc))
        
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--batch_size",
                      type=int,
                      default=32,
                      metavar="N",
                      help="Batch size for training (default: 128)")
    parser.add_argument("--learning_rate", 
                      type=float,  
                      default=0.1,
                      metavar="N",
                      help='Initial learning rate')
    parser.add_argument("--optimizer", 
                      type=str, 
                      default='adam',
                      metavar="N",
                      help='optimizer')
    parsed_args, _ = parser.parse_known_args()
    main(parsed_args)



Overwriting tfjobheart.py


Create the docker file

In [ ]:
FROM python:3.8
WORKDIR /schema
RUN pip install -U joblib pandas tensorflow-data-validation
RUN pip install -U joblib pandas tensorflow-data-validation ipython tensorflow 
COPY schemag.py /schema
ENTRYPOINT ["python", "schemag.py"] 

FROM tensorflow/tensorflow:2.4.0
RUN pip install tensorflow_datasets pandas scikit-learn keras
COPY tfjobheartdisease.py /
ENTRYPOINT ["python", "/tfjobheart.py", "--batch_size", "64", "--learning_rate", "0.1", "--optimizer", "adam"]




In [ ]:
%run $TRAINER_FILE --optimizer 'adam'

In [ ]:
 
%%writefile $KUBERNETES_FILE
apiVersion: "kubeflow.org/v1"
kind: "TFJob"
metadata:
  name: "hrtd"
  namespace: ekemini # your-user-namespace
spec:
  cleanPodPolicy: None
  tfReplicaSpecs:
    Worker:
      replicas: 2
      restartPolicy: OnFailure
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          containers:
          - name: tensorflow
            # modify this property if you would like to use a custom image
            image: mavencodevv/tfjob_heart:v.0.1
            command:
                - "python"
                - "/tfjobheart.py"
                - "--batch_size=64"
                - "--learning_rate=0.1"
                - "--optimizer=adam"



In [ ]:

%%capture tf_output --no-stderr
! kubectl create -f $KUBERNETES_FILE
TF_JOB = get_resource(tf_output)

In [ ]:
! kubectl describe $TF_JOB

In [ ]:
! kubectl get pods -l job-name=hrtd

In [ ]:
! kubectl logs -f hrtd-worker-0

In [ ]:
! kubectl delete $TF_JOB

In [ ]:
#! kubectl -n ekemini logs -f hrtd